In [1]:
import os
import json
from tqdm import tqdm
import time
import torch
import numpy as np
import cv2
from PIL import Image
from torchvision import transforms
from pathlib import Path
from collections import defaultdict

print("="*70)
print("🔍 DETR & GroundingDINO Evaluation on RSUD Dataset")
print("="*70)

# ----------------------------
# GPU Check
# ----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\nDevice: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# ----------------------------
# RSUD Dataset Configuration
# ----------------------------
BASE_PATH = Path(r"F:\skills-copilot-codespaces-vscode\thesis\rsuddataset\rsud20k")
VAL_IMAGE_DIR = BASE_PATH / "images" / "val"
VAL_LABEL_DIR = BASE_PATH / "labels" / "val"
OUTPUT_DIR = Path(r"F:\skills-copilot-codespaces-vscode\thesis\checkpoints\detr_gdino_eval")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# RSUD class names (13 classes)
CLASS_NAMES = [
    'Dilarang Berhenti',      # 0
    'Dilarang Parkir',        # 1
    'Dilarang Masuk',         # 2
    'Bahaya',                 # 3
    'Lampu Lalu Lintas Merah',# 4
    'Batas Kecepatan',        # 5
    'Wajib',                  # 6
    'Larangan Belok',         # 7
    'Zona Pejalan Kaki',      # 8
    'Petunjuk Arah',          # 9
    'Rambu Informasi',        # 10
    'Hati-hati',              # 11
    'Zona Khusus'             # 12
]
NUM_CLASSES = len(CLASS_NAMES)

# Model paths
DETR_WEIGHTS = Path(r"F:\skills-copilot-codespaces-vscode\thesis\checkpoints\detr_best.pth")
GDINO_DIR = Path(r"F:\skills-copilot-codespaces-vscode\thesis\all code\gdino")

# Evaluation settings
VIS_EVERY_N = 50          # Save visualization every N images
SCORE_THRES = 0.25        # Detection confidence threshold
IOU_THRESHOLD = 0.5       # IoU threshold for matching predictions to ground truth

print(f"\n📁 Dataset:")
print(f"  Images: {VAL_IMAGE_DIR}")
print(f"  Labels: {VAL_LABEL_DIR}")
print(f"  Classes: {NUM_CLASSES}")
print(f"  Output: {OUTPUT_DIR}")

# ----------------------------
# Helper Functions
# ----------------------------
def yolo_to_xyxy(yolo_box, img_w, img_h):
    """Convert YOLO [x_center, y_center, width, height] (normalized) to [x1, y1, x2, y2] (absolute)"""
    x_center, y_center, width, height = yolo_box
    x1 = (x_center - width / 2) * img_w
    y1 = (y_center - height / 2) * img_h
    x2 = (x_center + width / 2) * img_w
    y2 = (y_center + height / 2) * img_h
    return [x1, y1, x2, y2]

def xyxy_to_xywh(box):
    """Convert [x1, y1, x2, y2] to [x, y, width, height]"""
    x1, y1, x2, y2 = box
    return [x1, y1, x2 - x1, y2 - y1]

def calculate_iou(box1, box2):
    """Calculate IoU between two boxes in [x1, y1, x2, y2] format"""
    x1_min, y1_min, x1_max, y1_max = box1
    x2_min, y2_min, x2_max, y2_max = box2
    
    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)
    
    inter_area = max(0, inter_x_max - inter_x_min) * max(0, inter_y_max - inter_y_min)
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)
    union_area = box1_area + box2_area - inter_area
    
    return inter_area / union_area if union_area > 0 else 0

def load_yolo_annotations(label_path, img_w, img_h):
    """Load YOLO format annotations and convert to [x1, y1, x2, y2] format"""
    annotations = []
    if not os.path.exists(label_path):
        return annotations
    
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 5:
                cls_id = int(parts[0])
                x_center, y_center, width, height = map(float, parts[1:5])
                box = yolo_to_xyxy([x_center, y_center, width, height], img_w, img_h)
                annotations.append({'class': cls_id, 'box': box})
    return annotations

# ----------------------------
# Load validation images
# ----------------------------
print("\n📂 Loading validation images...")
val_images = sorted([f for f in os.listdir(VAL_IMAGE_DIR) if f.endswith(('.jpg', '.png'))])
print(f"  Found {len(val_images)} validation images")

# ----------------------------
# Image preprocessing for models
# ----------------------------
def prepare_image_for_detr(img_bgr):
    """Prepare image for DETR model"""
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    pil = Image.fromarray(img_rgb)
    # DETR expects 800x800
    transform = transforms.Compose([
        transforms.Resize((800, 800)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    return transform(pil).to(device)

# ----------------------------
# Check if models exist
# ----------------------------
detr_available = DETR_WEIGHTS.exists()
gdino_available = GDINO_DIR.exists() and (GDINO_DIR / "groundingdino").exists()

print(f"\n🤖 Model Availability:")
print(f"  DETR: {'✅ Available' if detr_available else '❌ Not found'} ({DETR_WEIGHTS})")
print(f"  GroundingDINO: {'✅ Available' if gdino_available else '❌ Not found'} ({GDINO_DIR})")

if not detr_available and not gdino_available:
    print("\n" + "="*70)
    print("⚠️ NO TRAINED MODELS FOUND")
    print("="*70)
    print("\n📝 Current Status:")
    print("  • DETR: Not trained yet")
    print("  • GroundingDINO: Not set up yet")
    print("\n🎯 What You Can Do:")
    print("\n  Option 1: Train DETR (NOT RECOMMENDED)")
    print("    • Run: detr.ipynb")
    print("    • Time: 75-150 hours of training")
    print("    • Result: Custom DETR model for RSUD dataset")
    print("\n  Option 2: Setup GroundingDINO (RECOMMENDED)")
    print("    • Already pretrained on many objects")
    print("    • Just needs proper folder setup")
    print("    • Zero-shot detection capability")
    print("\n  Option 3: Use YOLO (BEST OPTION)")
    print("    • Already trained: runs/detect/rsud20k_yolo11/weights/best.pt")
    print("    • Command: yolo val model=F:/skills-copilot-codespaces-vscode/thesis/runs/detect/rsud20k_yolo11/weights/best.pt data=F:/skills-copilot-codespaces-vscode/thesis/rsuddataset/rsud20k/images/data.yaml")
    print("    • Fast, accurate, and ready to use!")
    print("\n" + "="*70)
    print("💡 For your thesis, YOLO is the best object detection model.")
    print("   Compare it with CNN, ViT, and DINOv2 classification models.")
    print("="*70)
else:
    # ----------------------------
    # Load DETR Model (if available)
    # ----------------------------
    detr_model = None
    if detr_available:
        print("\n🔄 Loading DETR model...")
        try:
            from transformers import DetrForObjectDetection, DetrImageProcessor
            
            # Load model
            detr_model = DetrForObjectDetection.from_pretrained(
                "facebook/detr-resnet-50",
                num_labels=NUM_CLASSES,
                ignore_mismatched_sizes=True
            )
            
            # Load trained weights
            checkpoint = torch.load(DETR_WEIGHTS, map_location=device)
            if 'model_state_dict' in checkpoint:
                detr_model.load_state_dict(checkpoint['model_state_dict'])
            else:
                detr_model.load_state_dict(checkpoint)
            
            detr_model.to(device)
            detr_model.eval()
            print("  ✅ DETR model loaded successfully")
        except Exception as e:
            print(f"  ❌ Failed to load DETR: {e}")
            detr_model = None
    
    # ----------------------------
    # Load GroundingDINO Model (if available)
    # ----------------------------
    gdino_model = None
    if gdino_available:
        print("\n🔄 Loading GroundingDINO model...")
        try:
            import sys
            sys.path.insert(0, str(GDINO_DIR))
            
            from groundingdino.util.inference import load_model, load_image, predict
            
            config_path = GDINO_DIR / "groundingdino" / "config" / "GroundingDINO_SwinT_OGC.py"
            weights_path = GDINO_DIR / "weights" / "groundingdino_swint_ogc.pth"
            
            if config_path.exists() and weights_path.exists():
                gdino_model = load_model(str(config_path), str(weights_path))
                print("  ✅ GroundingDINO model loaded successfully")
            else:
                print(f"  ❌ GroundingDINO files not found")
                print(f"     Config: {config_path}")
                print(f"     Weights: {weights_path}")
        except Exception as e:
            print(f"  ❌ Failed to load GroundingDINO: {e}")
            gdino_model = None
    
    if detr_model is None and gdino_model is None:
        print("\n⚠️ No models could be loaded.")
        print("Evaluation cannot proceed without at least one model.")
    else:
        # ----------------------------
        # Run Evaluation
        # ----------------------------
        print("\n" + "="*70)
        print("🔍 Running Evaluation...")
        print("="*70)
        
        # Storage for results
        detr_results = {
            'predictions': [],
            'tp': 0, 'fp': 0, 'fn': 0,
            'total_gt': 0,
            'class_tp': [0] * NUM_CLASSES,
            'class_fp': [0] * NUM_CLASSES,
            'class_fn': [0] * NUM_CLASSES,
            'class_gt': [0] * NUM_CLASSES
        }
        
        gdino_results = {
            'predictions': [],
            'tp': 0, 'fp': 0, 'fn': 0,
            'total_gt': 0,
            'class_tp': [0] * NUM_CLASSES,
            'class_fp': [0] * NUM_CLASSES,
            'class_fn': [0] * NUM_CLASSES,
            'class_gt': [0] * NUM_CLASSES
        }
        
        vis_dir = OUTPUT_DIR / "visualizations"
        vis_dir.mkdir(exist_ok=True)
        
        print(f"\n Processing {len(val_images)} images...")
        
        for idx, img_name in enumerate(tqdm(val_images)):
            img_path = VAL_IMAGE_DIR / img_name
            label_path = VAL_LABEL_DIR / img_name.replace('.jpg', '.txt').replace('.png', '.txt')
            
            # Load image
            img_bgr = cv2.imread(str(img_path))
            if img_bgr is None:
                continue
            h, w = img_bgr.shape[:2]
            
            # Load ground truth
            gt_annotations = load_yolo_annotations(label_path, w, h)
            detr_results['total_gt'] += len(gt_annotations)
            gdino_results['total_gt'] += len(gt_annotations)
            for ann in gt_annotations:
                detr_results['class_gt'][ann['class']] += 1
                gdino_results['class_gt'][ann['class']] += 1
            
            # ----------------------------
            # DETR Inference
            # ----------------------------
            detr_detections = []
            if detr_model is not None:
                try:
                    input_tensor = prepare_image_for_detr(img_bgr).unsqueeze(0)
                    
                    with torch.no_grad():
                        outputs = detr_model(input_tensor)
                    
                    # Get predictions
                    logits = outputs.logits[0]  # [num_queries, num_classes]
                    boxes = outputs.pred_boxes[0]  # [num_queries, 4] in cxcywh format
                    
                    # Convert to probabilities
                    probs = logits.softmax(-1)
                    scores, labels = probs.max(-1)
                    
                    # Filter by score and convert boxes
                    for score, label, box in zip(scores, labels, boxes):
                        if score > SCORE_THRES and label < NUM_CLASSES:
                            # Convert from cxcywh (normalized) to xyxy (absolute)
                            cx, cy, bw, bh = box.cpu().numpy()
                            x1 = (cx - bw/2) * w
                            y1 = (cy - bh/2) * h
                            x2 = (cx + bw/2) * w
                            y2 = (cy + bh/2) * h
                            
                            detr_detections.append({
                                'class': int(label),
                                'box': [x1, y1, x2, y2],
                                'score': float(score)
                            })
                    
                    detr_results['predictions'].append({
                        'image': img_name,
                        'detections': detr_detections,
                        'ground_truth': gt_annotations
                    })
                except Exception as e:
                    print(f"\n  ⚠️ DETR error on {img_name}: {e}")
            
            # ----------------------------
            # GroundingDINO Inference
            # ----------------------------
            gdino_detections = []
            if gdino_model is not None:
                try:
                    # Create text prompt from class names
                    text_prompt = " . ".join(CLASS_NAMES) + " ."
                    
                    # Load image for GroundingDINO
                    from groundingdino.util.inference import load_image, predict
                    image_source, image_transformed = load_image(str(img_path))
                    
                    # Run inference
                    boxes, logits, phrases = predict(
                        model=gdino_model,
                        image=image_transformed,
                        caption=text_prompt,
                        box_threshold=SCORE_THRES,
                        text_threshold=0.25,
                        device=device
                    )
                    
                    # Convert boxes (normalized cxcywh) to absolute xyxy
                    for box, logit, phrase in zip(boxes, logits, phrases):
                        cx, cy, bw, bh = box.cpu().numpy()
                        x1 = (cx - bw/2) * w
                        y1 = (cy - bh/2) * h
                        x2 = (cx + bw/2) * w
                        y2 = (cy + bh/2) * h
                        
                        # Map phrase to class
                        try:
                            class_id = CLASS_NAMES.index(phrase)
                        except ValueError:
                            class_id = 0  # default
                        
                        gdino_detections.append({
                            'class': class_id,
                            'box': [x1, y1, x2, y2],
                            'score': float(logit)
                        })
                    
                    gdino_results['predictions'].append({
                        'image': img_name,
                        'detections': gdino_detections,
                        'ground_truth': gt_annotations
                    })
                except Exception as e:
                    print(f"\n  ⚠️ GroundingDINO error on {img_name}: {e}")
            
            # ----------------------------
            # Match predictions to ground truth (for DETR)
            # ----------------------------
            if detr_model is not None and len(detr_detections) > 0:
                matched_gt = set()
                for det in detr_detections:
                    matched = False
                    for gt_idx, gt in enumerate(gt_annotations):
                        if gt_idx in matched_gt:
                            continue
                        if det['class'] == gt['class']:
                            iou = calculate_iou(det['box'], gt['box'])
                            if iou >= IOU_THRESHOLD:
                                detr_results['tp'] += 1
                                detr_results['class_tp'][det['class']] += 1
                                matched_gt.add(gt_idx)
                                matched = True
                                break
                    if not matched:
                        detr_results['fp'] += 1
                        detr_results['class_fp'][det['class']] += 1
                
                # Count false negatives (unmatched ground truth)
                fn_count = len(gt_annotations) - len(matched_gt)
                detr_results['fn'] += fn_count
            
            # ----------------------------
            # Match predictions to ground truth (for GroundingDINO)
            # ----------------------------
            if gdino_model is not None and len(gdino_detections) > 0:
                matched_gt = set()
                for det in gdino_detections:
                    matched = False
                    for gt_idx, gt in enumerate(gt_annotations):
                        if gt_idx in matched_gt:
                            continue
                        if det['class'] == gt['class']:
                            iou = calculate_iou(det['box'], gt['box'])
                            if iou >= IOU_THRESHOLD:
                                gdino_results['tp'] += 1
                                gdino_results['class_tp'][det['class']] += 1
                                matched_gt.add(gt_idx)
                                matched = True
                                break
                    if not matched:
                        gdino_results['fp'] += 1
                        gdino_results['class_fp'][det['class']] += 1
                
                fn_count = len(gt_annotations) - len(matched_gt)
                gdino_results['fn'] += fn_count
            
            # ----------------------------
            # Visualization
            # ----------------------------
            if (idx % VIS_EVERY_N) == 0:
                vis_img = img_bgr.copy()
                
                # Draw ground truth (green)
                for gt in gt_annotations:
                    x1, y1, x2, y2 = map(int, gt['box'])
                    cv2.rectangle(vis_img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(vis_img, f"GT:{CLASS_NAMES[gt['class']][:10]}", 
                               (x1, max(15, y1-5)), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1)
                
                # Draw DETR predictions (blue)
                if detr_model is not None:
                    for det in detr_detections[:10]:  # Top 10
                        x1, y1, x2, y2 = map(int, det['box'])
                        cv2.rectangle(vis_img, (x1, y1), (x2, y2), (255, 0, 0), 2)
                        cv2.putText(vis_img, f"D:{det['score']:.2f}", 
                                   (x1, max(30, y1-20)), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 0, 0), 1)
                
                # Draw GroundingDINO predictions (red)
                if gdino_model is not None:
                    for det in gdino_detections[:10]:  # Top 10
                        x1, y1, x2, y2 = map(int, det['box'])
                        cv2.rectangle(vis_img, (x1, y1), (x2, y2), (0, 0, 255), 2)
                        cv2.putText(vis_img, f"G:{det['score']:.2f}", 
                                   (x1, max(45, y1-35)), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 1)
                
                cv2.imwrite(str(vis_dir / f"vis_{idx:04d}.jpg"), vis_img)
        
        # ----------------------------
        # Calculate Metrics
        # ----------------------------
        print("\n" + "="*70)
        print("📊 EVALUATION RESULTS")
        print("="*70)
        
        def calculate_metrics(results, model_name):
            tp = results['tp']
            fp = results['fp']
            fn = results['fn']
            
            precision = tp / (tp + fp) if (tp + fp) > 0 else 0
            recall = tp / (tp + fn) if (tp + fn) > 0 else 0
            f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
            
            print(f"\n--- {model_name} ---")
            print(f"  Total Ground Truth: {results['total_gt']}")
            print(f"  True Positives: {tp}")
            print(f"  False Positives: {fp}")
            print(f"  False Negatives: {fn}")
            print(f"  Precision: {precision*100:.2f}%")
            print(f"  Recall: {recall*100:.2f}%")
            print(f"  F1-Score: {f1:.4f}")
            
            # Per-class metrics
            print(f"\n  Per-Class Results:")
            for i in range(NUM_CLASSES):
                if results['class_gt'][i] > 0:
                    class_precision = results['class_tp'][i] / (results['class_tp'][i] + results['class_fp'][i]) if (results['class_tp'][i] + results['class_fp'][i]) > 0 else 0
                    class_recall = results['class_tp'][i] / results['class_gt'][i] if results['class_gt'][i] > 0 else 0
                    print(f"    {CLASS_NAMES[i][:20]:<20}: P={class_precision*100:5.1f}% R={class_recall*100:5.1f}% (GT={results['class_gt'][i]})")
            
            return {'precision': precision, 'recall': recall, 'f1': f1}
        
        if detr_model is not None:
            detr_metrics = calculate_metrics(detr_results, "DETR")
        
        if gdino_model is not None:
            gdino_metrics = calculate_metrics(gdino_results, "GroundingDINO")
        
        # ----------------------------
        # Save Results
        # ----------------------------
        print("\n" + "="*70)
        print("💾 Saving Results...")
        print("="*70)
        
        if detr_model is not None:
            with open(OUTPUT_DIR / "detr_results.json", "w") as f:
                json.dump(detr_results, f, indent=2)
            print(f"  ✅ DETR results saved to {OUTPUT_DIR / 'detr_results.json'}")
        
        if gdino_model is not None:
            with open(OUTPUT_DIR / "gdino_results.json", "w") as f:
                json.dump(gdino_results, f, indent=2)
            print(f"  ✅ GroundingDINO results saved to {OUTPUT_DIR / 'gdino_results.json'}")
        
        print(f"  ✅ Visualizations saved to {vis_dir}")
        
        print("\n" + "="*70)
        print("✅ Evaluation Complete!")
        print("="*70)

🔍 DETR & GroundingDINO Evaluation on RSUD Dataset

Device: cuda
GPU: NVIDIA GeForce RTX 3060

📁 Dataset:
  Images: F:\skills-copilot-codespaces-vscode\thesis\rsuddataset\rsud20k\images\val
  Labels: F:\skills-copilot-codespaces-vscode\thesis\rsuddataset\rsud20k\labels\val
  Classes: 13
  Output: F:\skills-copilot-codespaces-vscode\thesis\checkpoints\detr_gdino_eval

📂 Loading validation images...
  Found 1004 validation images

🤖 Model Availability:
  DETR: ❌ Not found (F:\skills-copilot-codespaces-vscode\thesis\checkpoints\detr_best.pth)
  GroundingDINO: ❌ Not found (F:\skills-copilot-codespaces-vscode\thesis\all code\gdino)

⚠️ No trained models found!

To train models:
  • DETR: Run detr.ipynb (WARNING: 75-150 hours training time)
  • GroundingDINO: Already pretrained, just needs proper setup

💡 RECOMMENDATION: Use YOLO model which is already trained!


SystemExit: 0

f:\skills-copilot-codespaces-vscode\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
